In [ ]:
from pynput import keyboard
import time
import logging
from cryptography.fernet import Fernet
import os

# Generate a key for encryption and store it securely (e.g., environment variable or config file)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Log file
log_file = "keystroke_timing.log"
encrypted_log_file = "keystroke_timing_encrypted.log"

# Initialize variables
last_release_time = None
last_key = None
press_time_dict = {}

def on_press(key):
    global last_release_time, last_key, press_time_dict
    press_time = time.time()

    if last_release_time is not None:
        flight_time = press_time - last_release_time
        logging.info(f"Flight Time: {flight_time:.4f} seconds")

    last_key = key
    press_time_dict[key] = press_time

def on_release(key):
    global last_release_time, last_key, press_time_dict
    release_time = time.time()

    if key in press_time_dict:
        hold_time = release_time - press_time_dict[key]
        logging.info(f"Hold Time: {hold_time:.4f} seconds")
        del press_time_dict[key]

    last_release_time = release_time

# Set up logging
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s: %(message)s')

# Encrypt the log file
def decrypt_log():
    try:
        # 从文件加载密钥
        with open('key.key', 'rb') as key_file:
            key = key_file.read()
        cipher_suite = Fernet(key)

        # 读取加密文件
        with open(encrypted_log_file, "rb") as f:
            encrypted_data = f.read()

        # 解密数据
        decrypted_data = cipher_suite.decrypt(encrypted_data)

        # 写入解密后的文件
        with open("decrypted_keystroke_timing.log", "wb") as f:
            f.write(decrypted_data)

        print("Decryption successful.")
    except Exception as e:
        print(f"Failed to decrypt log file: {e}")


# Start the keylogger
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    try:
        listener.join()
    except KeyboardInterrupt:
        encrypt_log()
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        encrypt_log()


In [10]:
import pandas as pd
import numpy as np

# Load and decrypt the log file
def decrypt_log():
    with open(log_file, "rb") as f:
        encrypted_data = f.read()
    decrypted_data = cipher_suite.decrypt(encrypted_data)
    with open("decrypted_keystroke_timing.log", "wb") as f:
        f.write(decrypted_data)

decrypt_log()

# Parse the log file
data = []
with open("decrypted_keystroke_timing.log", "r") as f:
    for line in f:
        if "Hold Time" in line or "Flight Time" in line:
            parts = line.strip().split(": ")
            data.append((parts[0], float(parts[1])))

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Type", "Time"])
df = df.pivot(columns="Type", values="Time")
df.dropna(inplace=True)

# Save preprocessed data
df.to_csv("keystroke_data.csv", index=False)

InvalidToken: 

In [ ]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Load preprocessed data
df = pd.read_csv("keystroke_data.csv")

# Train Isolation Forest model
model = IsolationForest(contamination=0.1, random_state=42)
df["Anomaly"] = model.fit_predict(df)

# Visualize anomalies
plt.scatter(df.index, df["Hold Time"], c=df["Anomaly"], cmap="coolwarm")
plt.title("Anomaly Detection in Keystroke Timing")
plt.xlabel("Keystroke Index")
plt.ylabel("Hold Time (seconds)")
plt.show()

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[["Hold Time", "Flight Time"]])

# Prepare sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:i+seq_length])
        ys.append(data[i+seq_length])
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

# Build LSTM model
model = Sequential([
    LSTM(50, activation="relu", input_shape=(seq_length, 2)),
    Dense(2)
])
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X, y, epochs=20, batch_size=32)

# Predict and detect anomalies
predictions = model.predict(X)
mse = np.mean(np.square(predictions - y), axis=1)
df["Anomaly"] = mse > np.percentile(mse, 95)

# Visualize anomalies
plt.scatter(df.index[seq_length:], df["Hold Time"][seq_length:], c=df["Anomaly"][seq_length:], cmap="coolwarm")
plt.title("Anomaly Detection in Keystroke Timing (LSTM)")
plt.xlabel("Keystroke Index")
plt.ylabel("Hold Time (seconds)")
plt.show()

ModuleNotFoundError: No module named 'tensorflow'